In [1]:
import tensorflow.keras as k

2025-08-13 20:29:40.471328: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 20:29:43.720064: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 20:29:45.712851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755106187.456100    6507 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755106187.850925    6507 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755106191.291678    6507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
def build_model(nx, layers, activations, lambatha, keep_prop):
    netowrk = k.models.Sequential()
    for i in range(len(layers)):
        if i == 0:
            netowrk.add(
                k.layers.Dense(
                    layers[i], activation=activations[i],
                    kernel_regularizer=k.regularizers.l2(lambatha),
                    input_shape=(nx, )))
        else:
            netowrk.add(
                k.layers.Dense(
                    layers[i], activation=activations[i],
                    kernel_regularizer=k.regularizers.l2(lambatha)
                )
            )
        
        if i != len(layers):
            k.layers.Dropout(1- keep_prop)
    return netowrk

In [4]:
network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, .95)
network.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 269,322 (1.03 MB)

 Trainable params: 269,322 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
def optimazer_model(network, alpha, beta_1, beta_2):
    optimazer = k.optimizers.Adam(alpha, beta_1, beta_2)
    network.compile(
        optimazer=optimazer, loss=k.losses.categorical_crossentropy, metrices=['accuracy']
    )

    


In [6]:
print(network.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.38617613911628723>, <tf.Tensor: shape=(), dtype=float32, numpy=0.25567635893821716>, <tf.Tensor: shape=(), dtype=float32, numpy=0.01891232095658779>]


In [7]:
def one_hot(labels, classes=None):
    one_hot = k.utils.to_categorical(labels, classes)
    return one_hot


In [11]:
(x_train, y_train), (x_test, y_test) = k.datasets.mnist.load_data()

y_train.shape
print(y_train[:10])
one_hot(y_train)

[5 0 4 1 9 2 1 3 1 4]


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [14]:
x_train.shape
x_train = x_train.reshape(x_train.shape[0], -1)
x_train.shape

(60000, 784)

In [20]:
labels = one_hot(y_train, 10)
print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [29]:
model = k.models.Sequential([
    k.layers.Dense(input_shape=(784, ),units=256,activation='relu', kernel_regularizer=k.regularizers.l2(.0001)),
    k.layers.Dropout(.05),
    k.layers.Dense(units=256, activation='relu', kernel_regularizer=k.regularizers.l2(.0001)),
    k.layers.Dropout(.05),
    k.layers.Dense(units=10, activation='softmax', kernel_regularizer=k.regularizers.l2(.0001))
])
model.compile(optimizer=k.optimizers.Adam(.001, .9, .999), loss=k.losses.categorical_crossentropy,
               metrics=['accuracy'])
model.fit(x_train, labels, epochs=5, batch_size=64, shuffle=True)


/home/ahmed/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.8755 - loss: 2.2256
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 34s 33ms/step - accuracy: 0.9304 - loss: 0.4201
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.9475 - loss: 0.2690
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - accuracy: 0.9571 - loss: 0.2181
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9608 - loss: 0.1950


In [30]:
x_train.shape

(60000, 784)

In [33]:
x_test.shape

(10000, 28, 28)

In [34]:
labels_valid = k.utils.to_categorical(y_test, 10)
x_test = x_test.reshape(10000, 784)

In [36]:
network = k.models.Sequential([
    k.layers.Dense(input_shape=(784,), units=256, activation='relu', kernel_regularizer=k.regularizers.l2(.0001)),
    k.layers.Dropout(.05),
    k.layers.Dense(units=256, activation='relu', kernel_regularizer=k.regularizers.l2(.0001)),
    k.layers.Dropout(.05),
    k.layers.Dense(units=10, activation='softmax'),
    ])
network.compile(optimizer=k.optimizers.Adam(.001, .9, .999), 
                metrics=['accuracy'], loss=k.losses.categorical_crossentropy)

early_stop = k.callbacks.EarlyStopping(monitor='val_accuracy', patience=6)

network.fit(x_train, labels, validation_data=(x_test, labels_valid), 
            epochs=30, batch_size=64, verbose=True,callbacks= early_stop)


/home/ahmed/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 35s 33ms/step - accuracy: 0.8822 - loss: 2.2074 - val_accuracy: 0.9324 - val_loss: 0.4624
Epoch 2/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9320 - loss: 0.3832 - val_accuracy: 0.9395 - val_loss: 0.2960
Epoch 3/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9486 - loss: 0.2566 - val_accuracy: 0.9538 - val_loss: 0.2600
Epoch 4/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 29ms/step - accuracy: 0.9564 - loss: 0.2122 - val_accuracy: 0.9580 - val_loss: 0.2317
Epoch 5/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.9570 - loss: 0.2098 - val_accuracy: 0.9597 - val_loss: 0.2106
Epoch 6/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 27s 29ms/step - accuracy: 0.9606 - loss: 0.2000 - val_accuracy: 0.9637 - val_loss: 0.2119
Epoch 7/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 29ms/step - accuracy: 0.9639 - loss: 0.1846 - val_accuracy: 0.9608 - val_loss: 0.2244
Epoch 8/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 35ms/step - accuracy: 0.9654 - loss: 0.1807 - 

In [22]:
y_train_onehot = k.utils.to_categorical(y_train, num_classes=10)